In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
import torchvision
from torchvision import datasets
from imutils import paths
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import os
import cv2
import numpy as np


In [ ]:
data_path ='./'
dataset = datasets.Caltech101(data_path, download = True)

Files already downloaded and verified


In [ ]:
image_paths = list(paths.list_images('./caltech101'))

data = []
labels = []
for img_path in tqdm(image_paths):
    label = img_path.split(os.path.sep)[-2]
    if label == "BACKGROUND_Google":
        continue
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    hist = cv2.calcHist([img], [0], None, [256], [0, 256])
    size = img.shape[0]*img.shape[1]
    hist = hist/size

    data.append(hist)
    labels.append(label)
    
data = np.array(data)
labels = np.array(labels)


100%|██████████| 9144/9144 [00:11<00:00, 792.96it/s] 


In [ ]:
lb = LabelEncoder()
labels = lb.fit_transform(labels)
print("Total number of classed: ",len(lb.classes_))

Total number of classed:  101


In [ ]:
train_ratio = 0.6
validation_ratio = 0.2
test_ratio = 0.2

x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=1 - train_ratio)
x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size=test_ratio/(test_ratio + validation_ratio)) 

x_train = x_train.reshape(len(x_train), 256)
x_val = x_val.reshape(len(x_val), 256)
x_test = x_test.reshape(len(x_test), 256)
print(len(x_train))
print(len(x_val))
print(len(x_test))


5206
1735
1736


##Yêu cầu 1: Áp dụng thuật toán KNN, tính Accuracy trên tập val và testvới các giá trị k lần lượt:1,2,3,4,5

In [ ]:
from sklearn import neighbors
from sklearn import metrics

In [ ]:
# Accuracy of test

best_score = 0
best_k = 0
for i in range(1,6):
    knn = neighbors.KNeighborsClassifier(n_neighbors=i)
    knn.fit(x_train, y_train)
    score = knn.score(x_val, y_val)
    if score > best_score:
        best_score = score
        best_k = i
    print(f'KNN with k={i}: {score}')
print(f"Best score = {best_score} with k = {best_k}")


KNN with k=1: 0.24092219020172911
KNN with k=2: 0.24495677233429394
KNN with k=3: 0.24322766570605187
KNN with k=4: 0.24956772334293947
KNN with k=5: 0.2559077809798271
Best score = 0.2559077809798271 with k = 5


In [ ]:
# Accuracy of validation

best_score = 0
best_k = 0
for i in range(1,6):
    knn = neighbors.KNeighborsClassifier(n_neighbors=i)
    knn.fit(x_train, y_train)
    score = knn.score(x_test, y_test)
    if score > best_score:
        best_score = score
        best_k = i
    print(f'KNN with k={i}: {score}')
print(f"Best score = {best_score} with k = {best_k}")


KNN with k=1: 0.23214285714285715
KNN with k=2: 0.2350230414746544
KNN with k=3: 0.2436635944700461
KNN with k=4: 0.24942396313364054
KNN with k=5: 0.2534562211981567
Best score = 0.2534562211981567 with k = 5


##Yêu cầu 2
###Áp dụng thuật toán SVC (với tham số C=1), 
###Tính Accuracy cho tập test
###Tính Precision của từng lớp (class)


In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score

In [ ]:
svc = SVC(verbose = 0 )
print(svc)
svc.fit(x_train,y_train)
score = svc.score(x_val, y_val)
print("Score Val: ", score)
score2 = svc.score(x_test, y_test)
print("Score Test: ",score2)

SVC(verbose=0)
Score Val:  0.26570605187319885
Score Test:  0.2597926267281106


In [ ]:
y_pred = svc.predict(x_test)
print("Acc",accuracy_score(y_pred, y_test))

Acc 0.2597926267281106


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           0       0.40      0.10      0.16       334
           1       0.74      0.14      0.24       414
           2       0.67      0.34      0.45        82
           3       0.82      0.41      0.54       313
           4       0.50      0.26      0.34        19
           5       0.83      0.52      0.64       261
           6       0.00      0.00      0.00         0
           7       0.00      0.00      0.00         0
           8       0.00      0.00      0.00         0
           9       0.00      0.00      0.00         0
          10       0.00      0.00      0.00         0
          11       0.10      1.00      0.18         1
          12       0.00      0.00      0.00         5
          13       0.21      0.10      0.13        31
          14       0.00      0.00      0.00         0
          15       0.00      0.00      0.00         0
          16       0.00      0.00      0.00         1
          17       0.00    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


##Yêu cầu 3
###Áp dụng thuật toán SVC (với tham số C=1), 
###Sử dụng chiến lược 1 vs 1:
####Tính Accuracy cho tập test
####Tính số support vector
####Tính và hiển thị ma trận Confusion


In [ ]:
def Cal_Acc(x_train, y_train, x_test, y_test, C=1, decision_function_shape='ovr'):
  svc = SVC(C=C, decision_function_shape=decision_function_shape)
  svc.fit(x_train, y_train)
  print(svc)
  acc = svc.score(x_test, y_test)
  predicts = svc.predict(x_test)
  n_support = svc.n_support_
  return acc, predicts, n_support

In [ ]:
acc, predicts, n_support = Cal_Acc(x_train, y_train, x_test, y_test, 1, 'ovo')


SVC(C=1, decision_function_shape='ovo')


In [ ]:
print("Accuracy for testing set is", acc)

Accuracy for testing set is 0.2597926267281106


In [ ]:
print("Number of support vectors is", sum(n_support))

Number of support vectors is 4808


In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
matrix = confusion_matrix(y_test, predicts)
matrix

array([[34, 47,  1, ...,  0,  0,  0],
       [20, 58,  0, ...,  0,  0,  0],
       [ 9,  4, 28, ...,  0,  0,  0],
       ...,
       [ 3,  9,  0, ...,  0,  0,  0],
       [ 0,  0,  1, ...,  0,  3,  0],
       [ 1,  5,  0, ...,  0,  0,  3]])

##Yêu cầu 4
###Áp dụng thuật toán SVC (với tham số C=1), 
###Sử dụng chiến lược 1 vs rest:
####Tính Accuracy cho tập test
####Tính số support vector
####Tính và hiển thị ma trận Confusion


In [ ]:
acc_4, predicts_4, n_support_4 = Cal_Acc(x_train, y_train, x_test, y_test, 1, 'ovo')
print("Accuracy for testing set is", acc_4)
print("Number of support vectors is", sum(n_support_4))
matrix = confusion_matrix(y_test, predicts_4)
matrix


SVC(C=1, decision_function_shape='ovo')
Accuracy for testing set is 0.2597926267281106
Number of support vectors is 4808


array([[34, 47,  1, ...,  0,  0,  0],
       [20, 58,  0, ...,  0,  0,  0],
       [ 9,  4, 28, ...,  0,  0,  0],
       ...,
       [ 3,  9,  0, ...,  0,  0,  0],
       [ 0,  0,  1, ...,  0,  3,  0],
       [ 1,  5,  0, ...,  0,  0,  3]])